**Ingesting production table from D365 Finance and Operations and extracting columns we need for M7 analysis.**

In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql.functions import *

df = spark.read.format("csv").load("Files/Tables/SupplyChain/ProductionControl/WorksheetHeader/ProdTable/PRODTABLE_00001.csv")
# df now is a Spark DataFrame containing CSV data from "Files/Tables/SupplyChain/ProductionControl/WorksheetHeader/ProdTable/PRODTABLE_00001.csv".
#display(df)

StatementMeta(, 49f16d4f-61c7-4cf3-8537-445cef915bc9, 3, Finished, Available)

Creating a new dataframe that only contains the columns we need from the production table

In [2]:
renamed_df = df.select(
                col("_c61").alias("ProjectID"),\
                col("_c53").alias("Production"),\
                col("_c6").alias("ItemNumber"),\
                col("_c7").alias("Name"),\
                col("_c15").alias("Quantity"),\
                col("_c85").alias("Created"),\
                col("_c39").alias("Pool"),\
                col("_c63").alias("ProjectCategory"),\
                col("_c0").alias("ID"),\
                col("_c3").alias("ImportDate"))

StatementMeta(, 49f16d4f-61c7-4cf3-8537-445cef915bc9, 4, Finished, Available)

Sorting by most recent production

In [3]:
renamed_df = renamed_df.sort(renamed_df['Production'].desc())

StatementMeta(, 49f16d4f-61c7-4cf3-8537-445cef915bc9, 5, Finished, Available)

Splitting the project id into project and subproject and rebuilding the dataframe

In [4]:
split_col = split(renamed_df['ProjectID'],'-')
renamed_df = renamed_df.withColumn('Proj', split_col.getItem(0))
renamed_df = renamed_df.withColumn('SubProj', split_col.getItem(1))

renamed_df = renamed_df.select(
                "Proj",\
                "SubProj",\
                "Production",\
                "ItemNumber",\
                "Name",\
                "Quantity",\
                "Created",\
                "Pool",\
                "ProjectCategory",\
                "ID",\
                "ImportDate")

#display(renamed_df)

StatementMeta(, 49f16d4f-61c7-4cf3-8537-445cef915bc9, 6, Finished, Available)

Create/overwrite a new table in datalake with the new dataframe

In [5]:
delta_table_path = "Tables/ProdTable" #fill in your delta table path 
renamed_df.write.format("delta").mode("overwrite").option("overwriteSchema","true").save(delta_table_path)

StatementMeta(, 49f16d4f-61c7-4cf3-8537-445cef915bc9, 7, Finished, Available)